In [1]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

from utils.load_data_for_eda import load_data
df = load_data('data/commodity_prices.csv')

# import warnings
# warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [2]:
df.head()

,Product_Type,Commodity,Arrival_Date,Market,Is_VFPCK,Season,Year,Modal_Price,Max_Price,Min_Price
0,Alsandikai|Alsandikai|FAQ,Alsandikai,2023-12-13,North Paravur,False,Winter,2023,5200.0,6000.0,5000.0
1,Alsandikai|Alsandikai|FAQ,Alsandikai,2023-12-14,North Paravur,False,Winter,2023,6200.0,6500.0,6000.0
2,Alsandikai|Alsandikai|FAQ,Alsandikai,2023-12-16,North Paravur,False,Winter,2023,4800.0,5600.0,4600.0
3,Alsandikai|Alsandikai|FAQ,Alsandikai,2023-12-18,North Paravur,False,Winter,2023,3500.0,4500.0,3000.0
4,Alsandikai|Alsandikai|FAQ,Alsandikai,2023-12-19,North Paravur,False,Winter,2023,5500.0,6000.0,5500.0


In [24]:
df[df['Product_Type'] == 'Banana|Nendra Bale|Large']['Market'].nunique()    

17

In [21]:
def has_yearly_variance(group, tol=0.05):
    yearly_means = group.groupby(['Market', 'Year'])['log_Modal_Price'].mean()
    return yearly_means.max() - yearly_means.min() > tol

In [22]:
def has_seasonal_variance(group, tol=0.05):
    seasonal_means = group.groupby(['Market', 'Season'])['log_Modal_Price'].mean()
    # Check if the difference between max and min seasonal mean exceeds tol
    return seasonal_means.max() - seasonal_means.min() > tol

In [23]:
using_mixedlm = True
for prod, group in df.groupby('Product_Type'):
     year_counts = group['Year'].nunique()
     group['log_Modal_Price'] = group['Modal_Price'].apply(lambda x: np.log(x) if x>0 else np.nan)
     if year_counts > 2 and group['Market'].nunique() > 2:
     #if year_counts > 1:
          if has_yearly_variance(group) and has_seasonal_variance(group):
                    if group['Season'].value_counts().shape[0] >= 2:
                         formula = smf.mixedlm('log_Modal_Price ~ C(Season) + Year',
                                             data=group,
                                             groups=group['Market'],
                                             re_formula='Year')
                         print("rich formula") 
                    else:
                         formula = smf.mixedlm('log_Modal_Price ~ Year',
                                             data=group,
                                             groups=group['Market'],
                                             re_formula='Year')
                         print("formula with no season")
          elif has_yearly_variance(group) and not has_seasonal_variance(group):
               formula = smf.mixedlm('log_Modal_Price ~ Year',
                                        data=group,
                                        groups=group['Market'],
                                        re_formula='Year')
               print("formula with no season")
          elif not has_yearly_variance(group) and has_seasonal_variance(group):
               formula = smf.mixedlm('log_Modal_Price ~ C(Season)',
                                        data=group,
                                        groups=group['Market'])
               print("formula with no year")
          else:
               formula = smf.mixedlm('log_Modal_Price ~ 1',
                                        data=group,
                                        groups=group['Market'])
               print("formula with no year and no season")
     elif year_counts == 2 and group['Market'].nunique() > 2:
          if has_seasonal_variance(group):
               if group['Season'].value_counts().shape[0] >= 2:
                    formula = smf.mixedlm('log_Modal_Price ~ C(Season)',
                                    data=group,
                                    groups=group['Market'])
                    print("formula with season only")
               else:
                    formula = smf.mixedlm('log_Modal_Price ~ 1',
                                    data=group,
                                    groups=group['Market'])
                    print("formula with no season and no year")
          else:
               formula = smf.mixedlm('log_Modal_Price ~ 1',
                                    data=group,
                                    groups=group['Market'])    
               print("formula with no season and no year")
     else:
          if has_yearly_variance(group) and has_seasonal_variance(group):
               if group['Season'].value_counts().shape[0] >= 2:
                    formula = smf.ols('log_Modal_Price ~ C(Season)', data=group)
                    using_mixedlm = False
                    print("OLS: only season effect")
               else:
                    formula = smf.ols('log_Modal_Price ~ 1', data=group)
                    using_mixedlm = False
                    print("OLS: constant mean only")
          elif has_seasonal_variance(group):
               formula = smf.ols('log_Modal_Price ~ C(Season)', data=group)
               using_mixedlm = False
               print("OLS: only season effect")
          else:
               formula = smf.ols('log_Modal_Price ~ 1', data=group)
               using_mixedlm = False
               print("OLS: constant mean only")
               
     if using_mixedlm:
          result = formula.fit(method="lbfgs", reml=False)
     else:  # OLS
          result = formula.fit()
     print(f'Product: {prod}')
     print(result.summary())         

       

OLS: only season effect
Product: Alsandikai|Alsandikai|FAQ
                            OLS Regression Results                            
Dep. Variable:        log_Modal_Price   R-squared:                       0.386
Model:                            OLS   Adj. R-squared:                  0.382
Method:                 Least Squares   F-statistic:                     97.89
Date:                Sat, 23 Aug 2025   Prob (F-statistic):           3.60e-49
Time:                        16:02:06   Log-Likelihood:                 193.01
No. Observations:                 471   AIC:                            -378.0
Df Residuals:                     467   BIC:                            -361.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("

Product: Amaranthus|Other|FAQ
                     Mixed Linear Model Regression Results
Model:                   MixedLM       Dependent Variable:       log_Modal_Price
No. Observations:        1160          Method:                   REML           
No. Groups:              3             Scale:                    0.0237         
Min. group size:         2             Log-Likelihood:           489.0055       
Max. group size:         589           Converged:                Yes            
Mean group size:         386.7                                                  
--------------------------------------------------------------------------------
                                Coef.   Std.Err.    z    P>|z|  [0.025   0.975] 
--------------------------------------------------------------------------------
Intercept                      -185.923   13.948 -13.330 0.000 -213.260 -158.586
C(Season)[T.Southwest Monsoon]    0.025    0.013   1.948 0.051   -0.000    0.050
C(Season)[T.Summer] 

/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)


Product: Amphophalus|Amphophalus|FAQ
                   Mixed Linear Model Regression Results
Model:                 MixedLM      Dependent Variable:      log_Modal_Price
No. Observations:      1895         Method:                  REML           
No. Groups:            8            Scale:                   0.0160         
Min. group size:       2            Log-Likelihood:          1192.8236      
Max. group size:       609          Converged:               Yes            
Mean group size:       236.9                                                
----------------------------------------------------------------------------
                                Coef.  Std.Err.    z    P>|z|  [0.025 0.975]
----------------------------------------------------------------------------
Intercept                      -11.364   10.627  -1.069 0.285 -32.192  9.464
C(Season)[T.Southwest Monsoon]   0.052    0.008   6.277 0.000   0.036  0.069
C(Season)[T.Summer]             -0.063    0.010  -6.418 0.0

...

/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("

Product: Tapioca|Tapioca|FAQ
                    Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      log_Modal_Price
No. Observations:       4285         Method:                  REML           
No. Groups:             10           Scale:                   0.0149         
Min. group size:        11           Log-Likelihood:          2821.0996      
Max. group size:        634          Converged:               Yes            
Mean group size:        428.5                                                
-----------------------------------------------------------------------------
                                Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
-----------------------------------------------------------------------------
Intercept                      -83.903    5.902 -14.217 0.000 -95.470 -72.337
C(Season)[T.Southwest Monsoon]  -0.120    0.005 -22.426 0.000  -0.131  -0.110
C(Season)[T.Summer]             -0.087    0.006 -13.586

/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2206: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2218: ConvergenceWarning: Gradient optimization failed, |grad| = 2567.798568
  warnings.warn(msg, ConvergenceWarning)
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parame

                            OLS Regression Results                            
Dep. Variable:        log_Modal_Price   R-squared:                       0.387
Model:                            OLS   Adj. R-squared:                  0.305
Method:                 Least Squares   F-statistic:                     4.729
Date:                Sat, 23 Aug 2025   Prob (F-statistic):             0.0256
Time:                        16:02:20   Log-Likelihood:                 1.8175
No. Observations:                  18   AIC:                             2.365
Df Residuals:                      15   BIC:                             5.036
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               8.3112    

/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/aswinbenny/Downloads/projects/commodity_pricing/.venv/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


In [ ]:
using_mixedlm = True
for prod, group in df.groupby('Product_Type'):
     year_counts = group['Year'].nunique()
     group['log_Modal_Price'] = group['Modal_Price'].apply(lambda x: np.log(x) if x>0 else np.nan)
     #if year_counts > 1 and group['Market'].nunique() > 1:
     if year_counts > 1:
          if has_yearly_variance(group) and has_seasonal_variance(group):
                    if group['Season'].value_counts().shape[0] >= 2:
                         formula = smf.mixedlm('log_Modal_Price ~ C(Season) + Year',
                                             data=group,
                                             groups=group['Market'],
                                             re_formula='Year')
                         print("rich formula") 
                    else:
                         formula = smf.mixedlm('log_Modal_Price ~ Year',
                                             data=group,
                                             groups=group['Market'],
                                             re_formula='Year')
                         print("formula with no season")
          elif has_yearly_variance(group) and not has_seasonal_variance(group):
               formula = smf.mixedlm('log_Modal_Price ~ Year',
                                        data=group,
                                        groups=group['Market'],
                                        re_formula='Year')
               print("formula with no season")
          elif not has_yearly_variance(group) and has_seasonal_variance(group):
               formula = smf.mixedlm('log_Modal_Price ~ C(Season)',
                                        data=group,
                                        groups=group['Market'])
               print("formula with no year")
          else:
               formula = smf.mixedlm('log_Modal_Price ~ 1',
                                        data=group,
                                        groups=group['Market'])
               print("formula with no year and no season")
     
     else:
          if has_seasonal_variance(group):
               if group['Season'].value_counts().shape[0] >= 2:
                    formula = smf.mixedlm('log_Modal_Price ~ C(Season)',
                                    data=group,
                                    groups=group['Market'])
                    print("formula with season only")
               else:
                    formula = smf.mixedlm('log_Modal_Price ~ 1',
                                    data=group,
                                    groups=group['Market'])
                    print("formula with no season and no year")
          else:
               formula = smf.mixedlm('log_Modal_Price ~ 1',
                                    data=group,
                                    groups=group['Market'])    
               print("formula with no season and no year")
     # if using_mixedlm:                
     result = formula.fit(method="lbfgs", reml=True)
     # if using_mixedlm:
     #      result = formula.fit(method="lbfgs", reml=False)
     # else:  # OLS
     #      result = formula.fit()
     print(f'Product: {prod}')
     print(result.summary())         

       

In [ ]:
df[df['Product_Type'] == 'Amla(Nelli Kai)|Amla|FAQ']['Market'].value_counts().shape

In [ ]:
df[df['Product_Type'] == 'Ashgourd|Ashgourd|FAQ']['Market'].value_counts().shape

In [ ]:
df[df['Product_Type'] == 'Arecanut(Betelnut/Supari)|Supari|FAQ']['Market'].value_counts().shape

In [ ]:
def prepare_for_mixedlm(df, min_years=2, min_obs=30):
    refined = []
    for prod, group in df.groupby("Product_Type"):
        # keep only if there’s enough obs
        if len(group) < min_obs:
            continue

        # check markets have at least min_years variation
        year_counts = group.groupby("Market")["Year"].nunique()
        if year_counts.max() < min_years:
            continue

        refined.append(group)

    return pd.concat(refined, axis=0)

df = prepare_for_mixedlm(df)

In [ ]:
df['Product_Type'].nunique(), 

In [ ]:
df[df['Product_Type'] == 'Amphophalus|Amphophalus|FAQ'].shape

In [ ]:
df['log_Modal_Price'] = df['Modal_Price'].apply(lambda x: np.log(x) if x > 0 else 0)
for Product_Type in df['Product_Type'].unique():
    print(f"Product Type: {Product_Type}")
    subset = df[df['Product_Type'] == Product_Type]
    model = smf.mixedlm('log_Modal_Price ~ C(Season) + Year',
                        data=subset,
                        groups=subset['Market'],
                        re_formula='Year')
    result = model.fit(method="lbfgs", reml=False)
    print(result.summary())
    print("\n" + "="*80 + "\n")

In [ ]:
m_ml   = m.fit(method="lbfgs", reml=False)  # model selection
m_reml = m.fit(method="lbfgs", reml=True)   # final interpretation

In [ ]:
df.dtypes

In [ ]:
pd.crosstab(df['Market'], df['Season'])

In [ ]:
pd.crosstab(df['Product_Type'], df['Market'])

In [ ]:
pd.crosstab(df['Market'], df['Year'])

In [ ]:
pd.crosstab(df['Product_Type'], df['Year'])

In [ ]:
model = smf.mixedlm(
    "Modal_Price ~ C(Product_Type) + C(Season) + Year",
    data=df,
    groups=df["Market"],
    re_formula="~Year"
)
result = model.fit(method="lbfgs", reml=True)


In [ ]:
result.summary()

**Observation:**  
Mixed-effects model results show clear, consistent price differences between markets, even after controlling for product type and arrival date.  
This suggests that market-level factors (e.g., supply chain, demand patterns, VFPCK vs. non-VFPCK operations) play a significant role in price variation.  
These coefficients provide a baseline-adjusted measure of each market’s typical pricing, which we will use to rank markets by relative price levels.

In [ ]:
# Get market ranking
fe_params = result.fe_params
market_ranking = fe_params.filter(like='C(Market)')
market_ranking.index = market_ranking.index.str.replace('C(Market)[T.', '').str.replace(']', '')
market_ranking['Aluva'] = 0  # Add Aluva as baseline
market_ranking.sort_values(ascending=True, inplace=True)
print(market_ranking)

We observe that most VFPCK markets tend to offer lower prices compared to non-VFPCK markets.  
Earlier seasonal analysis showed that VFPCK prices also fluctuate with seasonality.  
From a consumer’s perspective, these lower prices are beneficial, but for farmers, they may reduce profit margins.  

Higher prices in urban or semi-urban markets like Angamaly and Ernakulam could be driven by stronger demand and higher willingness to pay, possibly amplified by transportation and logistics costs.  
However, to draw a stronger conclusion, we would need to map each market’s location, classify them as urban/rural, and examine their surrounding population and demand patterns.